In [3]:
import json
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import SGD, RMSprop

import balloonml

In [5]:
config  ={}
config['top_model_weights_path'] = 'disdat2_v1.h5'
config['train_data_dir'] = '../data'
config['validation_data_dir'] = '../validation'

config['bottleneck_train'] = 'bottleneck_features_train.npy'
config['bottleneck_validation'] = 'bottleneck_features_validation.npy'

# True if bottleneck features are not available yet, 
# False if they are and can be lodaded
calculate_bottleneck = False 

config['epochs'] = 50
config['batch_size'] = 32

# config['optimizer']=RMSprop(lr=0.0001, decay=1E-5)
config['optimizer']='sgd'
config['lr']=0.0005
config['decay']=5E-10

# config['optimizer']=RMSprop()
config['output']='disdat-110-sgd-decay5'

with open(config['output']+'.config', 'w') as fp:
    json.dump(config, fp)

## Saving or loading bottleneck features

In [6]:
if calculate_bottleneck:
    model = VGG16(include_top=False, weights='imagenet')
    train_data = balloonml.bottleneck(model,config['train_data_dir'],config['bottleneck_train'])
    validation_data = balloonml.bottleneck(model,config['validation_data_dir'],config['bottleneck_validation'])
else:
    train_data = np.load(config['bottleneck_train'])
    validation_data = np.load(config['bottleneck_validation'])

## Running model

In [ ]:
if config['optimizer']=='sgd':
    optimizer=SGD(lr=config['lr'], decay=config['decay'])
elif config['optimizer']=='rmsprop':
    optimizer=RMSprop(lr=config['lr'], decay=config['decay'])

history = balloonml.train_top(
    train_data, 
    validation_data,
    train_data_dir=config['train_data_dir'], 
    validation_data_dir=config['validation_data_dir'],
    optimizer=optimizer,
    batch_size = config['batch_size']
    epochs=config['epochs'],
    output=config['output'])
balloonml.plotResult(history)

## Prediction

In [ ]:
image_path = 'test_images/keyboard.jpg'
img=mpimg.imread(image_path)
plt.imshow(img)
print("I think this is a "+balloonml.predict(image_path, output+'.h5')+".")